# Creating Category Dimenion Table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_catalog.gold.dim_category
(
  category_id STRING NOT NULL PRIMARY KEY,
  product STRING,
  sub_product STRING,
  issue STRING,
  sub_issue STRING
)
USING DELTA
LOCATION 'abfss://gold@rcmadls10dev.dfs.core.windows.net/dim_category';

In [0]:
%sql
-- select category_id, count(category_id) from databricks_catalog.gold.dim_category 
-- group by category_id
-- having count(category_id) > 1
-- ;

-- drop table databricks_catalog.gold.dim_category;

-- truncate table databricks_catalog.gold.dim_category;

In [0]:
%sql
WITH deduped_source AS (
  SELECT
    category_id,
    FIRST(product) as product,
    FIRST(sub_product) as sub_product,
    FIRST(issue) as issue,
    FIRST(sub_issue) as sub_issue
  FROM databricks_catalog.silver.complaints
  WHERE category_id IS NOT NULL
  GROUP BY category_id
)

MERGE INTO databricks_catalog.gold.dim_category TARGET
USING deduped_source SOURCE
ON TARGET.category_id = SOURCE.category_id
WHEN MATCHED THEN
  UPDATE SET 
    TARGET.product = SOURCE.product,
    TARGET.sub_product = SOURCE.sub_product,
    TARGET.issue = SOURCE.issue,
    TARGET.sub_issue = SOURCE.sub_issue
WHEN NOT MATCHED THEN
  INSERT (category_id, product, sub_product, issue, sub_issue)
  VALUES (SOURCE.category_id, SOURCE.product, SOURCE.sub_product, SOURCE.issue, SOURCE.sub_issue)

In [0]:
%sql
select * from databricks_catalog.gold.dim_category;